In [78]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [79]:
from skforecast.ForecasterBaseline import ForecasterLastEquivalentDate
from skforecast.model_selection import backtesting_forecaster
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from pandas.tseries import offsets
from skforecast import __version__
print(__version__)


0.10.1


In [80]:
# simulate a time series
n = 365
time = pd.date_range(start='2000-01-01', periods=n, freq='d')
y = np.arange(n)
df = pd.DataFrame({'time': time, 'y': y})
df = df.set_index('time')
df = df.asfreq('d')
df

,y
time,
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
...,...
2000-12-26,360
2000-12-27,361
2000-12-28,362


In [81]:
forecaster = ForecasterLastEquivalentDate(offset=10, n_equivalent_dates=2)
forecaster.fit(y=df['y'])
forecaster

ForecasterLastEquivalentDate 
Offset: 10 
Number of equivalent dates: 2 
Aggregation function: mean 
Window size: 20 
Training range: [Timestamp('2000-01-01 00:00:00'), Timestamp('2000-12-30 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Creation date: 2023-10-27 23:44:25 
Last fit date: 2023-10-27 23:44:25 
Skforecast version: 0.10.1 
Python version: 3.11.4 
Forecaster id: None 

In [82]:
forecaster.last_window

time
2000-12-11    345
2000-12-12    346
2000-12-13    347
2000-12-14    348
2000-12-15    349
2000-12-16    350
2000-12-17    351
2000-12-18    352
2000-12-19    353
2000-12-20    354
2000-12-21    355
2000-12-22    356
2000-12-23    357
2000-12-24    358
2000-12-25    359
2000-12-26    360
2000-12-27    361
2000-12-28    362
2000-12-29    363
2000-12-30    364
Freq: D, Name: y, dtype: int32

In [83]:
forecaster.predict(steps=15)

2000-12-31    350.0
2001-01-01    351.0
2001-01-02    352.0
2001-01-03    353.0
2001-01-04    354.0
2001-01-05    355.0
2001-01-06    356.0
2001-01-07    357.0
2001-01-08    358.0
2001-01-09    359.0
2001-01-10    350.0
2001-01-11    351.0
2001-01-12    352.0
2001-01-13    353.0
2001-01-14    354.0
Freq: D, Name: pred, dtype: float64

In [84]:
np.arange(-5, -0)

array([-5, -4, -3, -2, -1])

In [85]:
metric, predictions_1 = backtesting_forecaster(
    forecaster=forecaster,
    y=df['y'],
    initial_train_size=forecaster.window_size,
    steps=9,
    metric='mean_absolute_error',
    verbose=False
)

  0%|          | 0/39 [00:00<?, ?it/s]

In [149]:
forecaster = ForecasterLastEquivalentDate(
    offset=offsets.BusinessDay(10),
    n_equivalent_dates=1
)
forecaster.fit(y=df['y'])
forecaster

ForecasterLastEquivalentDate 
Offset: <10 * BusinessDays> 
Number of equivalent dates: 1 
Aggregation function: mean 
Window size: 13 
Training range: [Timestamp('2000-01-01 00:00:00'), Timestamp('2000-12-30 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Creation date: 2023-10-28 00:13:44 
Last fit date: 2023-10-28 00:13:44 
Skforecast version: 0.10.1 
Python version: 3.11.4 
Forecaster id: None 

In [150]:
forecaster.last_window

time
2000-12-18    352
2000-12-19    353
2000-12-20    354
2000-12-21    355
2000-12-22    356
2000-12-23    357
2000-12-24    358
2000-12-25    359
2000-12-26    360
2000-12-27    361
2000-12-28    362
2000-12-29    363
2000-12-30    364
Freq: D, Name: y, dtype: int32

In [152]:
forecaster.predict(steps=15)

AttributeError: 'numpy.int32' object has no attribute 'to_numpy'

In [147]:
df['y']

time
2000-01-01      0
2000-01-02      1
2000-01-03      2
2000-01-04      3
2000-01-05      4
             ... 
2000-12-26    360
2000-12-27    361
2000-12-28    362
2000-12-29    363
2000-12-30    364
Freq: D, Name: y, Length: 365, dtype: int32

In [148]:
metric, predictions_2 = backtesting_forecaster(
    forecaster=forecaster,
    y=df['y'],
    initial_train_size=50,
    steps=9,
    metric='mean_absolute_error',
    verbose=False,
    n_jobs=1
)

  0%|          | 0/35 [00:00<?, ?it/s]

KeyError: "[Timestamp('2000-02-01 00:00:00')] not in index"

In [91]:
pd.testing.assert_frame_equal(predictions_1, predictions_2)

In [106]:
# simulate time series with dayly frequency
n = 365
time = pd.date_range(start='2000-01-01', periods=n, freq='D')
y = np.arange(n)
y = pd.Series(y, index=time)
offset = offsets.Day(n=10)
y

2000-01-01      0
2000-01-02      1
2000-01-03      2
2000-01-04      3
2000-01-05      4
             ... 
2000-12-26    360
2000-12-27    361
2000-12-28    362
2000-12-29    363
2000-12-30    364
Freq: D, Length: 365, dtype: int32

In [117]:
time - pd.TimedeltaIndex([offset]*n)

DatetimeIndex(['1999-12-22', '1999-12-23', '1999-12-24', '1999-12-25',
               '1999-12-26', '1999-12-27', '1999-12-28', '1999-12-29',
               '1999-12-30', '1999-12-31',
               ...
               '2000-12-11', '2000-12-12', '2000-12-13', '2000-12-14',
               '2000-12-15', '2000-12-16', '2000-12-17', '2000-12-18',
               '2000-12-19', '2000-12-20'],
              dtype='datetime64[ns]', length=365, freq=None)

In [52]:
a = pd.date_range(start='2000-01-01', end='2000-01-03', freq='D').to_numpy()
pd.DatetimeIndex(a)

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03'], dtype='datetime64[ns]', freq=None)

In [ ]:
# Substract 10 days to the las value of the time series index and add one unit of the time series frequency
y.index[-1] - pd.DateOffset(days=10) 

Timestamp('2000-12-20 00:00:00')

In [ ]:
y.index[-1] - pd.DateOffset(days=10) + 2*y.index.freq

Timestamp('2000-12-22 00:00:00')

In [ ]:
from skforecast.utils import expand_index

last_window = y.tail(20)
last_window_index = y.index
last_window_values = y.to_numpy()
n_equivalent_dates = 2
print(last_window_index[-1])
steps = 15
index_freq = y.index.freq


predictions_index = expand_index(index=last_window_index,steps=steps)
equivalent_dates = [
                predictions_index[0] - offset + n * index_freq
                for n
                in np.arange(steps)[:len(last_window)]
            ]
equivalent_dates = np.tile(
                        equivalent_dates,
                        int(np.ceil(steps/len(equivalent_dates)))
                    )
equivalent_dates = equivalent_dates[:steps]

#substract off set element whise
equivalent_dates = [
    [
        date - offset * n
        for n
        in np.arange(n_equivalent_dates)
    ]
    for date
    in equivalent_dates
]
print(equivalent_dates)

equivalent_values = [
    last_window.loc[dates].to_numpy()
    for dates
    in equivalent_dates
]
equivalent_values


2000-12-30 00:00:00
[[Timestamp('2000-12-21 00:00:00'), Timestamp('2000-12-11 00:00:00')], [Timestamp('2000-12-22 00:00:00'), Timestamp('2000-12-12 00:00:00')], [Timestamp('2000-12-23 00:00:00'), Timestamp('2000-12-13 00:00:00')], [Timestamp('2000-12-24 00:00:00'), Timestamp('2000-12-14 00:00:00')], [Timestamp('2000-12-25 00:00:00'), Timestamp('2000-12-15 00:00:00')], [Timestamp('2000-12-26 00:00:00'), Timestamp('2000-12-16 00:00:00')], [Timestamp('2000-12-27 00:00:00'), Timestamp('2000-12-17 00:00:00')], [Timestamp('2000-12-28 00:00:00'), Timestamp('2000-12-18 00:00:00')], [Timestamp('2000-12-29 00:00:00'), Timestamp('2000-12-19 00:00:00')], [Timestamp('2000-12-30 00:00:00'), Timestamp('2000-12-20 00:00:00')], [Timestamp('2000-12-31 00:00:00'), Timestamp('2000-12-21 00:00:00')], [Timestamp('2001-01-01 00:00:00'), Timestamp('2000-12-22 00:00:00')], [Timestamp('2001-01-02 00:00:00'), Timestamp('2000-12-23 00:00:00')], [Timestamp('2001-01-03 00:00:00'), Timestamp('2000-12-24 00:00:00')],

KeyError: "[Timestamp('2000-12-31 00:00:00')] not in index"

In [ ]:

predictions = np.apply_along_axis(
    func1d=np.mean,
    axis=1,
    arr=equivalent_values
)

predictions

array([350., 351., 352., 353., 354., 355., 356., 357., 358., 359., 350.,
       351., 352., 353., 354.])

In [ ]:
equivalent_values = [
    
]

KeyError: "None of [Index([(2000-12-21 00:00:00, 2000-12-11 00:00:00),\n       (2000-12-22 00:00:00, 2000-12-12 00:00:00),\n       (2000-12-23 00:00:00, 2000-12-13 00:00:00),\n       (2000-12-24 00:00:00, 2000-12-14 00:00:00),\n       (2000-12-25 00:00:00, 2000-12-15 00:00:00),\n       (2000-12-26 00:00:00, 2000-12-16 00:00:00),\n       (2000-12-27 00:00:00, 2000-12-17 00:00:00),\n       (2000-12-28 00:00:00, 2000-12-18 00:00:00),\n       (2000-12-29 00:00:00, 2000-12-19 00:00:00),\n       (2000-12-30 00:00:00, 2000-12-20 00:00:00),\n       (2000-12-21 00:00:00, 2000-12-11 00:00:00),\n       (2000-12-22 00:00:00, 2000-12-12 00:00:00),\n       (2000-12-23 00:00:00, 2000-12-13 00:00:00),\n       (2000-12-24 00:00:00, 2000-12-14 00:00:00),\n       (2000-12-25 00:00:00, 2000-12-15 00:00:00)],\n      dtype='object')] are in the [index]"

In [ ]:
#substract off set element whise
[
    [
        date - offset * n
        for n
        in np.arange(n_equivalent_dates)
    ]
    for date
    in equivalent_dates
]

TypeError: unsupported operand type(s) for -: 'list' and 'pandas._libs.tslibs.offsets.Day'

In [ ]:
np.arange(steps)[:len(last_window)]

365